# 神经网络模型：

In [140]:
import torch

class Para:
    # tensor_board_log_dir = 'runs/exp0'
    feature_column_start_name = 'ep_ratio_ttm'
    feature_column_end_name = 'BR'

    # 模型设置
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    classification = 2 # 2, 3

    # 权重
    cross_weight = list()
    if classification == 3:
        cross_weight = [1.0, 1.0 ,1.0]
    elif classification == 2:
        cross_weight = [1.0, 1.0]
    elif classification == 5:
        cross_weight = [1.0, 1.0, 1.0, 1.0, 1.0]

    batch_size = 16
    lr = 1e-3
    drop = 0.5
    epochs = 100

    # 数据集设置
    month_in_sample = range(0, 1)
    # month_test = range(36, 48)

    percent_cv = 0.1 # 10% cross validation

    data_path = 'data/sk_space_1d_rate_20d_12-21_pre'


    seed = 2022
    torch.manual_seed(seed)

    info_str0 = data_path[5:7]+'_'+'c'+str(classification)+'_s'+str(percent_cv)
    info_str1 = '_b'+str(batch_size)+'_lr'+str(lr)+'_d'+str(drop)+'_e'+str(epochs)
    info_str = info_str0 + info_str1

    save_model_path = 'models/'+'model_'+info_str+'.pth'

para = Para()
print(para.info_str)
print(para.save_model_path)
print(torch.cuda.is_available())
print(torch.cuda.device_count())
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))
    print(torch.cuda.current_device())

sk_c2_s0.1_b16_lr0.001_d0.5_e100
models/model_sk_c2_s0.1_b16_lr0.001_d0.5_e100.pth
True
1
GeForce MX150
0


### 构建训练集

In [141]:
import numpy as np
import pandas as pd

data_in_sample = None
for i_month in para.month_in_sample:
    file_name = para.data_path + '/' + str(i_month) + '.csv'
    data_curr_month = pd.read_csv(file_name)

    data_curr_month = data_curr_month.dropna(axis=0)

    data_curr_month.insert(loc=0, column='return_bin', value=np.nan)

    data_curr_month.loc[data_curr_month['yield_rate']>0, 'return_bin'] = 0
    data_curr_month.loc[data_curr_month['yield_rate']<=0, 'return_bin'] = 1

    if i_month == para.month_in_sample[0]:
        data_in_sample = data_curr_month
    else:
        data_in_sample = pd.concat([data_in_sample, data_curr_month])
        # data_in_sample = data_in_sample.append(data_curr_month)

data_in_sample

,return_bin,order_book_id,board_type,sector_code,month,date,yield_rate,ep_ratio_ttm,pb_ratio_ttm,sp_ratio_ttm,...,RSI10,SY,BIAS20,VOL30,VOL60,VOL120,VOLT20,VOLT60,AR,BR
0,1.0,600376.XSHG,MainBoard,RealEstate,0,2012-01-04,-0.141230,-1.524457,0.643879,-1.212339,...,0.538128,1.224282,0.934854,0.186499,0.250212,0.183803,-0.329742,-0.644819,2.494509,1.125687
1,0.0,600376.XSHG,MainBoard,RealEstate,1,2012-01-05,0.253379,-1.500471,0.570059,-1.201655,...,0.330607,1.224282,0.417093,0.217907,0.263290,0.180967,-0.340798,-0.635916,2.293300,0.968585
2,0.0,600376.XSHG,MainBoard,RealEstate,2,2012-01-06,0.749352,-1.503539,0.579287,-1.203022,...,0.650744,1.224282,0.428205,0.211118,0.248994,0.178574,-0.338216,-0.627350,1.857657,0.677275
3,0.0,600376.XSHG,MainBoard,RealEstate,3,2012-01-09,0.065030,-1.536937,0.683865,-1.217897,...,0.743422,1.224282,0.985446,0.229623,0.245450,0.177982,-0.305186,-0.609930,1.762400,0.498572
4,1.0,600376.XSHG,MainBoard,RealEstate,4,2012-01-10,-0.213278,-1.564457,0.776140,-1.230155,...,0.650744,1.224282,1.428671,0.267379,0.249163,0.186044,-0.233927,-0.574659,2.099490,0.715619
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2384,0.0,600376.XSHG,MainBoard,RealEstate,2406,2021-11-29,1.462886,1.657715,-1.359218,3.640367,...,-0.065573,-0.882930,0.052101,-0.646078,-0.707748,-0.780226,-1.025460,-0.470301,-1.132233,-1.416920
2385,0.0,600376.XSHG,MainBoard,RealEstate,2407,2021-11-30,1.647394,1.683664,-1.364536,3.640367,...,-0.143578,-0.882930,-0.060633,-0.653847,-0.708004,-0.780349,-1.031112,-0.460610,-0.648826,-0.976654
2386,0.0,600376.XSHG,MainBoard,RealEstate,2408,2021-12-01,1.530037,1.632091,-1.353900,3.640367,...,0.447523,-0.180526,0.148899,-0.655539,-0.708258,-0.779669,-1.028788,-0.454117,-0.258064,-0.658283
2387,0.0,600376.XSHG,MainBoard,RealEstate,2409,2021-12-02,1.484967,1.565309,-1.339718,3.599144,...,0.917404,-0.180526,0.403522,-0.652971,-0.707706,-0.778008,-0.992226,-0.452088,0.532003,-0.119379


In [142]:
X_in_sample = data_in_sample.loc[:, para.feature_column_start_name: para.feature_column_end_name]
y_in_sample = data_in_sample.loc[:, 'return_bin']

from sklearn.model_selection import train_test_split

X_train, X_cv, y_train, y_cv = train_test_split(X_in_sample, y_in_sample, test_size=para.percent_cv, shuffle=False) # True, random_state=para.seed)
# X_train, X_cv, y_train, y_cv = train_test_split(X_in_sample, y_in_sample, test_size=para.percent_cv, shuffle=False)

X_train

,ep_ratio_ttm,pb_ratio_ttm,sp_ratio_ttm,MACD_DIFF,MACD_DEA,MACD_HIST,RSI10,SY,BIAS20,VOL30,VOL60,VOL120,VOLT20,VOLT60,AR,BR
0,-1.524457,0.643879,-1.212339,0.965225,0.697316,0.990854,0.538128,1.224282,0.934854,0.186499,0.250212,0.183803,-0.329742,-0.644819,2.494509,1.125687
1,-1.500471,0.570059,-1.201655,0.896273,0.745902,0.605944,0.330607,1.224282,0.417093,0.217907,0.263290,0.180967,-0.340798,-0.635916,2.293300,0.968585
2,-1.503539,0.579287,-1.203022,0.840603,0.773069,0.333522,0.650744,1.224282,0.428205,0.211118,0.248994,0.178574,-0.338216,-0.627350,1.857657,0.677275
3,-1.536937,0.683865,-1.217897,0.889795,0.805143,0.395923,0.743422,1.224282,0.985446,0.229623,0.245450,0.177982,-0.305186,-0.609930,1.762400,0.498572
4,-1.564457,0.776140,-1.230155,1.008477,0.855748,0.631619,0.650744,1.224282,1.428671,0.267379,0.249163,0.186044,-0.233927,-0.574659,2.099490,0.715619
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2145,-0.243384,-1.009849,1.050885,-0.230040,-0.284554,0.118673,-0.350847,-0.882930,-0.151494,-0.772571,-0.707969,-0.698921,-1.008562,-0.377083,-0.982731,0.303978
2146,-0.239937,-1.011728,1.055094,-0.231966,-0.276739,0.087384,-0.350847,-1.585334,-0.178863,-0.772718,-0.708999,-0.699805,-1.011489,-0.369160,-0.830266,0.417762
2147,-0.190519,-1.038037,1.115448,-0.273718,-0.279262,-0.044115,-1.053790,-1.585334,-0.519861,-0.770928,-0.709570,-0.700236,-0.934769,-0.355120,-0.675094,0.659123
2148,-0.186903,-1.039916,1.119863,-0.306850,-0.288245,-0.126274,-0.928130,-1.585334,-0.524438,-0.771887,-0.712030,-0.701534,-0.872603,-0.341957,-0.733134,0.823248


In [143]:
y_train

0       1.0
1       0.0
2       0.0
3       0.0
4       1.0
       ... 
2145    1.0
2146    1.0
2147    1.0
2148    1.0
2149    1.0
Name: return_bin, Length: 2150, dtype: float64

In [144]:
data_train = pd.concat([X_train, y_train], axis=1)
data_cv = pd.concat([X_cv, y_cv], axis=1)
data_train

,ep_ratio_ttm,pb_ratio_ttm,sp_ratio_ttm,MACD_DIFF,MACD_DEA,MACD_HIST,RSI10,SY,BIAS20,VOL30,VOL60,VOL120,VOLT20,VOLT60,AR,BR,return_bin
0,-1.524457,0.643879,-1.212339,0.965225,0.697316,0.990854,0.538128,1.224282,0.934854,0.186499,0.250212,0.183803,-0.329742,-0.644819,2.494509,1.125687,1.0
1,-1.500471,0.570059,-1.201655,0.896273,0.745902,0.605944,0.330607,1.224282,0.417093,0.217907,0.263290,0.180967,-0.340798,-0.635916,2.293300,0.968585,0.0
2,-1.503539,0.579287,-1.203022,0.840603,0.773069,0.333522,0.650744,1.224282,0.428205,0.211118,0.248994,0.178574,-0.338216,-0.627350,1.857657,0.677275,0.0
3,-1.536937,0.683865,-1.217897,0.889795,0.805143,0.395923,0.743422,1.224282,0.985446,0.229623,0.245450,0.177982,-0.305186,-0.609930,1.762400,0.498572,0.0
4,-1.564457,0.776140,-1.230155,1.008477,0.855748,0.631619,0.650744,1.224282,1.428671,0.267379,0.249163,0.186044,-0.233927,-0.574659,2.099490,0.715619,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2145,-0.243384,-1.009849,1.050885,-0.230040,-0.284554,0.118673,-0.350847,-0.882930,-0.151494,-0.772571,-0.707969,-0.698921,-1.008562,-0.377083,-0.982731,0.303978,1.0
2146,-0.239937,-1.011728,1.055094,-0.231966,-0.276739,0.087384,-0.350847,-1.585334,-0.178863,-0.772718,-0.708999,-0.699805,-1.011489,-0.369160,-0.830266,0.417762,1.0
2147,-0.190519,-1.038037,1.115448,-0.273718,-0.279262,-0.044115,-1.053790,-1.585334,-0.519861,-0.770928,-0.709570,-0.700236,-0.934769,-0.355120,-0.675094,0.659123,1.0
2148,-0.186903,-1.039916,1.119863,-0.306850,-0.288245,-0.126274,-0.928130,-1.585334,-0.524438,-0.771887,-0.712030,-0.701534,-0.872603,-0.341957,-0.733134,0.823248,1.0


In [145]:
from torch.utils.data import TensorDataset
import torch

X_train_ndarray = X_train.values
y_train_ndarray = y_train.values

train_dataset = TensorDataset(torch.from_numpy(X_train_ndarray).type(torch.FloatTensor), torch.from_numpy(y_train_ndarray).type(torch.LongTensor))

# for X_train_temp, y_train_temp in train_dataset:
#     print(X_train_temp, y_train_temp)
#     print(X_train_temp.dtype, y_train_temp.dtype)

In [146]:
X_cv_ndarray = X_cv.values
y_cv_ndarray = y_cv.values

cv_dataset = TensorDataset(torch.from_numpy(X_cv_ndarray).type(torch.FloatTensor), torch.from_numpy(y_cv.values).type(torch.LongTensor))

# for X_cv_temp, y_cv_temp in cv_dataset:
#     print(X_cv_temp, y_cv_temp)
#     print(X_cv_temp.dtype, y_cv_temp.dtype)

In [147]:
from torch.utils.data import DataLoader


train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=para.batch_size,
    shuffle=True
)

cv_dataloader = DataLoader(
    dataset=cv_dataset,
    batch_size=para.batch_size,
    shuffle=True
)

### 构建测试集

In [148]:
# data_test = None
# for i_month in para.month_test:
#
#     file_name = para.data_path + '/' + str(i_month) + '.csv'
#     data_curr_month = pd.read_csv(file_name)
#
#     data_curr_month = data_curr_month.dropna(axis=0)
#
#     data_curr_month = label_data(data=data_curr_month, percent_select=para.percent_select)
#
#     if i_month == para.month_test[0]:
#         data_test = data_curr_month
#     else:
#         data_test = pd.concat([data_test, data_curr_month])
#         # data_test = data_test.append(data_curr_month)
#
# X_test = data_test.loc[:, para.feature_column_start_name: para.feature_column_end_name]
# y_test = data_test.loc[:, 'return_bin']

In [149]:
# from torch.utils.data import TensorDataset
# import torch
#
#
# X_test_ndarray = X_test.values
# y_test_ndarray = y_test.values
#
# test_dataset = TensorDataset(torch.from_numpy(X_test_ndarray).type(torch.FloatTensor), torch.from_numpy(y_test_ndarray).type(torch.LongTensor))

In [150]:
# from torch.utils.data import DataLoader
#
#
# test_dataloader = DataLoader(
#     dataset=test_dataset,
#     batch_size=para.batch_size,
#     shuffle=True,
# )

### 构建神经网络

In [151]:
from my_utils.model_class import MLP

model = MLP(in_nums=len(X_train.columns), out_nums=para.classification, drop_p=para.drop)
# to device
model = model.to(device=para.device)
print(model)

MLP(
  (linear_stack): Sequential(
    (0): Linear(in_features=16, out_features=16, bias=True)
    (1): Dropout(p=0.5, inplace=False)
    (2): ReLU()
    (3): Linear(in_features=16, out_features=8, bias=True)
    (4): Dropout(p=0.5, inplace=False)
    (5): ReLU()
    (6): Linear(in_features=8, out_features=2, bias=True)
  )
)


### 训练与测试

In [152]:
import torchmetrics


def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    train_loss, correct = 0, 0

    # initialize metric
    train_precision = torchmetrics.Precision(average='none', num_classes=para.classification).to(device=para.device)

    for batch, (X, y) in enumerate(dataloader):

        # to device
        X = X.to(device=para.device)
        y = y.to(device=para.device)

        # compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        train_loss += loss_fn(pred, y).item()
        correct += (pred.argmax(1) == y).type(torch.float).sum().item()

        # metric on current batch
        train_precision(pred.argmax(1), y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # if batch % 10 == 0:
        #     loss, current = loss.item(), batch * len(X)
        #     print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

    train_loss /= num_batches
    correct /= size
    print(f"Train Error: \n    Accuracy: {(100*correct):>0.1f}%, Avg loss: {train_loss:>8f} \n")

    # metric on all batches using custom accumulation
    total_precision = train_precision.compute()
    print("Precision of every train dataset class: ", total_precision)
    print()

    return correct, train_loss, total_precision


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # initialize metric
    test_precision = torchmetrics.Precision(average='none', num_classes=para.classification).to(device=para.device)

    with torch.no_grad():
        for X, y in dataloader:

            # to device
            X = X.to(device=para.device)
            y = y.to(device=para.device)

            # compute prediction and loss
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

            # metric on current batch
            test_precision(pred.argmax(1), y)


    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n    Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

    # metric on all batches using custom accumulation
    total_precision = test_precision.compute()
    print("Precision of every test dataset class: ", total_precision)
    print()

    return correct, test_loss, total_precision

In [153]:
def select_df_to_dataloader(df: pd.DataFrame, select: int) -> DataLoader:

    df = df[df['return_bin'] == select]

    df_dataset = TensorDataset(
        torch.from_numpy(df.loc[:, para.feature_column_start_name: para.feature_column_end_name].values).type(torch.FloatTensor),
        torch.from_numpy(df.loc[:, 'return_bin'].values).type(torch.LongTensor))

    df_dataloader = DataLoader(
        dataset=df_dataset,
        batch_size=para.batch_size,
        shuffle=True,
    )

    return df_dataloader

In [154]:
# temp2_dataloader = select_df_to_dataloader(df=data_cv, select=2)
temp1_dataloader = select_df_to_dataloader(df=data_cv, select=1)
temp0_dataloader = select_df_to_dataloader(df=data_cv, select=0)

In [155]:
from torch import nn
import time
from torch.utils.tensorboard import SummaryWriter

# 计时
time_start = time.time()

# writer = SummaryWriter(para.tensor_board_log_dir)
writer = SummaryWriter()

# 损失函数
loss_fn = nn.CrossEntropyLoss()
# to device
loss_fn = loss_fn.to(device=para.device)

# 优化器
optimizer = torch.optim.Adam(model.parameters(), lr=para.lr)

epochs = para.epochs
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")

    model.train()
    accuracy_train, loss_train, precision_train = train_loop(train_dataloader, model, loss_fn, optimizer)
    model.eval()
    accuracy_cv, loss_cv, precision_cv = test_loop(cv_dataloader, model, loss_fn)

    # accuracy2 = test_loop(temp2_dataloader, model, loss_fn)
    # print('#')
    # accuracy1 = test_loop(temp1_dataloader, model, loss_fn)
    # accuracy0 = test_loop(temp0_dataloader, model, loss_fn)

    # 写入 tensorboard
    if para.classification == 2:

        writer.add_scalars(main_tag=para.info_str+'_evaluation/cv',
                           tag_scalar_dict={
                               'accuracy': accuracy_cv,
                               'precision0': precision_cv[0],
                               'precision1': precision_cv[1]},
                           global_step=t)

        writer.add_scalars(main_tag=para.info_str+'_evaluation/train',
                           tag_scalar_dict={
                               'accuracy': accuracy_train,
                               'precision0': precision_train[0],
                               'precision1': precision_train[1]},
                           global_step=t)

    elif para.classification == 3:

        writer.add_scalars(main_tag=para.info_str+'_evaluation/cv',
                           tag_scalar_dict={
                               'accuracy': accuracy_cv,
                               'precision0': precision_cv[0],
                               'precision1': precision_cv[1],
                               'precision2': precision_cv[2]},
                           global_step=t)

        writer.add_scalars(main_tag=para.info_str+'_evaluation/train',
                           tag_scalar_dict={
                               'accuracy': accuracy_train,
                               'precision0': precision_train[0],
                               'precision1': precision_train[1],
                               'precision2': precision_train[2]},
                           global_step=t)

    elif para.classification == 5:

        writer.add_scalars(main_tag=para.info_str+'_evaluation/cv',
                           tag_scalar_dict={
                               'accuracy': accuracy_cv,
                               'precision0': precision_cv[0],
                               'precision1': precision_cv[1],
                               'precision2': precision_cv[2],
                               'precision3': precision_cv[3],
                               'precision4': precision_cv[4]},
                           global_step=t)

        writer.add_scalars(main_tag=para.info_str+'_evaluation/train',
                           tag_scalar_dict={
                               'accuracy': accuracy_train,
                               'precision0': precision_train[0],
                               'precision1': precision_train[1],
                               'precision2': precision_train[2],
                               'precision3': precision_train[3],
                               'precision4': precision_train[4]},
                           global_step=t)

    writer.add_scalars(main_tag=para.info_str+'_loss/cv',
                       tag_scalar_dict={
                           'loss': loss_cv},
                       global_step=t)

    writer.add_scalars(main_tag=para.info_str+'_loss/train',
                       tag_scalar_dict={
                           'loss': loss_train},
                       global_step=t)
    writer.flush()

    time_end = time.time()
    print('Time cost = %fs' % (time_end - time_start))
    print()

writer.close()

print("Done!")

Epoch 1
-------------------------------
Train Error: 
    Accuracy: 51.0%, Avg loss: 0.693752 

Precision of every train dataset class:  tensor([0.4395, 0.5368], device='cuda:0')

Test Error: 
    Accuracy: 64.0%, Avg loss: 0.679700 

Precision of every test dataset class:  tensor([0.7037, 0.6321], device='cuda:0')

Time cost = 2.086793s

Epoch 2
-------------------------------
Train Error: 
    Accuracy: 53.4%, Avg loss: 0.690061 

Precision of every train dataset class:  tensor([0.4510, 0.5427], device='cuda:0')

Test Error: 
    Accuracy: 62.8%, Avg loss: 0.677834 

Precision of every test dataset class:  tensor([1.0000, 0.6147], device='cuda:0')

Time cost = 4.186877s

Epoch 3
-------------------------------
Train Error: 
    Accuracy: 53.7%, Avg loss: 0.688645 

Precision of every train dataset class:  tensor([0.4454, 0.5426], device='cuda:0')

Test Error: 
    Accuracy: 59.4%, Avg loss: 0.677572 

Precision of every test dataset class:  tensor([0.0000, 0.5941], device='cuda:0')



## 保存模型

In [156]:
torch.save(model.state_dict(), para.save_model_path)

print('Finish save model!')

Finish save model!


## captum

In [157]:
# # captum
# from captum.attr import IntegratedGradients
#
# ig = IntegratedGradients(model)

In [158]:
# temp = cv_dataloader.dataset.tensors[0]
# temp.requires_grad_()
# attr, delta = ig.attribute(temp,target=1, return_convergence_delta=True)
# attr = attr.detach().numpy()

In [159]:
# # Helper method to print importances and visualize distribution
# def visualize_importances(feature_names, importances, title="Average Feature Importances", plot=True, axis_title="Features"):
#     print(title)
#     for i in range(len(feature_names)):
#         print(feature_names[i], ": ", '%.3f'%(importances[i]))
#     y_pos = (np.arange(len(feature_names)))
#     if plot:
#         plt.figure(figsize=(20,6))
#         plt.barh(y_pos, importances, align='center')
#         plt.yticks(y_pos, feature_names)
#         plt.ylabel(axis_title)
#         plt.grid(axis='y')
#         plt.title(title)
# visualize_importances(feature_names=X_cv.columns.values.tolist(), importances=np.mean(attr, axis=0))

In [160]:
# X_cv.columns.values.tolist()

In [161]:
# loss = nn.CrossEntropyLoss()
# input = torch.Tensor(
#     [[-0.0441,  0.0773],
#     [-0.0781, -0.1772],
#     [-0.1319, -0.0432],
#     [-0.0714, -0.1261],
#     [-0.0806, -0.1370],
#     [-0.1730, -0.1472],
#     [-0.0350, -0.0507],
#     [-0.1149, -0.2248]])
# # input = input.reshape(-1,4)
# target = torch.Tensor([0, 1, 1, 0, 0, 0, 0, 0]).type(torch.LongTensor)
# print(input.dtype)
# print(target.dtype)
# output = loss(input, target)
# print(input, target, output)

In [162]:
# # Example of target with class indices
# loss = nn.CrossEntropyLoss()
# input = torch.randn(3, 5, requires_grad=True)
# target = torch.Tensor([1,4,1]).type(torch.LongTensor)
# output = loss(input, target)
# print(input,target,output)

In [163]:
# loss = nn.BCEWithLogitsLoss()
# input = torch.Tensor([0.5, 0.4, 0.3])
# target = torch.Tensor([0])
# output = loss(input, target)
# print(input, target, output)